## Flux comparions
This notebook compares the predictions of the R. opacus PD630 GSM with the fluxes from 13C-metabolic flux analysis (13C-MFA)

### Methods: 
<ol>
<li></li>
<li></li>
</ol>

### Imports

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import cobra
import pandas as pd
import numpy as np
from edd_utils import login, export_study, export_metadata

### Load model 

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml")
model

Name,ropacus_annotated_curated
Memory address,0x07f1765f9c810
Number of metabolites,1583
Number of reactions,2385
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [3]:
# edd_study_slug = 'input-test-study'
# edd_server = 'public-edd.agilebiofoundry.org'
# user= 'garrettroell'
# session = login(edd_server=edd_server, user=user)
# # df = utils.load_study(edd_study_slug=edd_study_slug,edd_server=edd_server,
# #   default_to_system_user=False)

In [4]:
# df = export_study(session, edd_study_slug, edd_server=edd_server)
# df

### Get 13C MFA measured fluxes for phenol

In [5]:
glucose_fluxes = pd.read_csv('../13C_flux_data/13C_glucose_flux_data.csv')
glucose_fluxes

,Pathway,Forward Reactions,Reverse Reactions,Reversed?,Reaction,Flux,90% Confidence Lower Bound,90% Confidence Upper Bound,\
0,Glucose Uptake,EX_glc__D_e,EX_glc__D_e_reverse_af641,NaN,Gluc.ext + ATP -> G6P,100.00,100.00,100.00,NaN
1,Glycolysis,PPGKr_reverse_3b2cc or HEX1,PPGKr or HEX1_reverse_25efa,False,G6P <-> F6P,-1.61,-2.09,1.42,NaN
2,Glycolysis,FBP or PFK_reverse_d24a6,FBP_reverse_bf2c9 or PFK,NaN,F6P + ATP -> FBP,0.00,0.00,1.91,NaN
3,Glycolysis,FBA,FBA_reverse_84806,False,FBP <-> DHAP + GAP,0.00,0.00,1.91,NaN
4,Glycolysis,TPI,TPI_reverse_c2c3b,False,DHAP <-> GAP,0.00,0.00,1.91,NaN
5,Glycolysis,GAPD and PGK_reverse_02696,GAPD_reverse_459c1 and PGK,NaN,GAP <-> 3PG + ATP + NADH,86.56,82.61,88.84,NaN
6,Glycolysis,( PGM_1 or PGM_reverse_fc9af ) and ENO,( PGM or PGM_1_reverse_02963 ) and ENO_reverse...,NaN,3PG <-> PEP,78.52,71.65,81.26,NaN
7,Glycolysis,FRUpts2pp or SBTpts or MNLpts or ACGApts or FR...,FRUpts2pp_reverse_55dac or SBTpts_reverse_74ed...,NaN,PEP <-> Pyr + ATP,19.02,-10.89,75.15,NaN
8,Pentose Phosphate Pathway,NaN,NaN,NaN,G6P <-> 6PG + NADPH,100.00,96.96,100.00,NaN
9,Pentose Phosphate Pathway,NaN,NaN,NaN,6PG <-> Ru5P + CO2 + NADPH,6.62,2.07,10.92,NaN


In [6]:
test_df = glucose_fluxes[glucose_fluxes['Pathway'] == 'Glycolysis']
test_df

,Pathway,Forward Reactions,Reverse Reactions,Reversed?,Reaction,Flux,90% Confidence Lower Bound,90% Confidence Upper Bound,\
1,Glycolysis,PPGKr_reverse_3b2cc or HEX1,PPGKr or HEX1_reverse_25efa,False,G6P <-> F6P,-1.61,-2.09,1.42,NaN
2,Glycolysis,FBP or PFK_reverse_d24a6,FBP_reverse_bf2c9 or PFK,NaN,F6P + ATP -> FBP,0.00,0.00,1.91,NaN
3,Glycolysis,FBA,FBA_reverse_84806,False,FBP <-> DHAP + GAP,0.00,0.00,1.91,NaN
4,Glycolysis,TPI,TPI_reverse_c2c3b,False,DHAP <-> GAP,0.00,0.00,1.91,NaN
5,Glycolysis,GAPD and PGK_reverse_02696,GAPD_reverse_459c1 and PGK,NaN,GAP <-> 3PG + ATP + NADH,86.56,82.61,88.84,NaN
6,Glycolysis,( PGM_1 or PGM_reverse_fc9af ) and ENO,( PGM or PGM_1_reverse_02963 ) and ENO_reverse...,NaN,3PG <-> PEP,78.52,71.65,81.26,NaN
7,Glycolysis,FRUpts2pp or SBTpts or MNLpts or ACGApts or FR...,FRUpts2pp_reverse_55dac or SBTpts_reverse_74ed...,NaN,PEP <-> Pyr + ATP,19.02,-10.89,75.15,NaN


### Run glucose flux FBA

In [39]:
with model:
    model.objective = 'Growth_Glucose'
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    model.medium = medium
    solution = model.optimize()
    
print(solution)

<Solution 0.793 at 0x7f181d1d0490>


In [41]:
type(solution.fluxes)

pandas.core.series.Series

In [46]:
for r in model.reactions:
    if solution.fluxes[r.id] > 8:
        print(r.id, r.name, solution.fluxes[r.id])

ATPS4rpp ATP synthase (four protons for one ATP) (periplasm) 43.17125454205829
CBFC2pp Cytochrome b6/f complex periplasm 20.0
CYO1bpp_syn Cytochrome c oxidase, synechocystis (2 protons) 20.0
ENO Enolase 15.906815592416013
F6PA Fructose 6-phosphate aldolase 8.903724302980127
GAPD Glyceraldehyde-3-phosphate dehydrogenase 16.81082623189666
GLCpts D-glucose transport via PEP:Pyr PTS 10.0
ICDHyr Isocitrate dehydrogenase (NADP) 28.471855855377367
NDH1_4pp Active co2 transporter facilitator (periplasm)  20.0
O2t O2 transport  diffusion  10.0
PDH Pyruvate dehydrogenase 10.38321261116513
PGI Glucose-6-phosphate isomerase 9.781806902332951
TPI Triose-phosphate isomerase 8.22606009371
EX_h2o_e R_EX_h2o_e 35.37966493387631
EX_h_e R_EX_h_e 11.241794525953939
EX_co2_e R_EX_co2_e 10.33016917776932


In [40]:
for r in solution.fluxes.sort_values(ascending=False)[:50]:
    print(r)

43.17125454205829
35.37966493387631
28.471855855377367
20.0
20.0
20.0
16.81082623189666
15.906815592416013
11.241794525953939
10.38321261116513
10.33016917776932
10.0
10.0
9.781806902332951
8.903724302980127
8.22606009371
7.587285931691979
7.587285931691979
7.314981831766829
7.314981831766829
5.458032818794115
4.473054496663836
4.466228854585141
4.466228854585141
4.466228854585141
3.4456914841860113
3.3532083881284254
2.0375155467337587
1.9831958198062978
1.6496793751143426
1.263176469301177
0.9404744725204562
0.9404744725204562
0.9404744725204562
0.904010639480646
0.904010639480646
0.904010639480646
0.8215278124855978
0.7933023386368248
0.7344609757963159
0.7344609757963159
0.731254926109863
0.6730099581607092
0.6556383516232241
0.6551948956159263
0.6545792930011441
0.6311677466084264
0.6107483029774272
0.6107483029774272
0.6107483029774272


In [47]:
model.reactions.get_by_id('PPGKr_reverse_3b2cc')

KeyError: 'PPGKr_reverse_3b2cc'

### Add Glucose FBA flux values to test_df 

In [25]:
for _, row in test_df.iterrows():
    reactions = row['Forward Reactions']
    flux = 0
    for x in [x.strip('() ') for x in reactions.split(' or ')]:
        and_split = [y.strip('() ') for y in x.split(' and ')]
        print(and_split)
        flux = [solution.fluxes[v] for v in and_split]
        print(flux)
    print()
        
        
        
        
# temp.add(frozenset(y.strip('() ') for y in x.split(' and ')))


#     for y in or_split:
#         print([y.strip('() ') for y in or_split.split(' and ')])
    
#     print(row['Forward Reactions'])
#     print(solution.fluxes[reaction_id])

['PPGKr_reverse_3b2cc']


KeyError: 'PPGKr_reverse_3b2cc'

### Get 13C MFA measured fluxes

In [ ]:
glucose_flux_path = "../13C_flux_data/13C_glucose_flux_data.csv"
measured_glucose_df = pd.read_csv(glucose_flux_path)
measured_glucose_df

Eliminate all rows that do not have a defined reactionID

In [ ]:
measured_glucose_df = measured_glucose_df[~pd.isnull(measured_glucose_df['Reaction ID'])]
measured_glucose_df

In [ ]:
measured_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
#     print(measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Reversed?'].values[0])
    measured_flux = measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Flux'].values[0] / 10
#     if measured_glucose_df[measured_glucose_df['Reaction ID'] == reaction_id]['Reversed?'].values[0]:
#         measured_flux = -1 * measured_flux
    measured_flux_dict[reaction_id] = measured_flux
measured_flux_dict

### Get FBA simulated fluxes

In [ ]:
simulated_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
    simulated_flux_dict[reaction_id] = solution.fluxes[reaction_id]

simulated_flux_dict

### Plot FBA vs measured fluxes

In [ ]:
measured_flux_list = list(measured_flux_dict.values())
simulated_flux_list = list(simulated_flux_dict.values())
labels = list(measured_flux_dict.keys())

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(measured_flux_list, simulated_flux_list)
# ax.plot(ax.get_xlim(), ax.get_ylim(), ls="--", c=".3")
x = np.linspace(*ax.get_xlim())
ax.plot(x, x, ls="--", c=".3")
for i in range(len(labels)):
    xy = (measured_flux_list[i]+.1,simulated_flux_list[i]+.1)
    ax.annotate(labels[i],xy)
plt.title(r'R. opacus 13C MFA vs FBA Fluxes', fontSize=20)
plt.xlabel(r'13C MFA flux measurement (mmol/gDW/h)')
plt.ylabel(r'FBA flux prediction (mmol/gDW/h)')
plt.show()

### Simulate with pfba

In [ ]:
pfba_solution = cobra.flux_analysis.pfba(model)
len(pfba_solution.fluxes)

In [ ]:
pfba_flux_dict = {}
measured_reaction_ids = list(measured_glucose_df['Reaction ID'])
for reaction_id in measured_reaction_ids:
    pfba_flux_dict[reaction_id] = pfba_solution.fluxes[reaction_id]

pfba_flux_dict

In [ ]:
measured_flux_list = list(measured_flux_dict.values())
simulated_flux_list = list(pfba_flux_dict.values())
labels = list(measured_flux_dict.keys())

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(measured_flux_list, simulated_flux_list)
x = np.linspace(*ax.get_xlim())
ax.plot(x, x, ls="--", c=".3")
for i in range(len(labels)):
    xy = (measured_flux_list[i]+.1,simulated_flux_list[i]+.1)
    ax.annotate(labels[i],xy)
plt.title(r'R. opacus 13C MFA vs FBA Fluxes', fontSize=20)
plt.xlabel(r'13C MFA flux measurement (mmol/gDW/h)')
plt.ylabel(r'pFBA flux prediction (mmol/gDW/h)')
plt.show()

### Simulate with geometric FBA

In [ ]:
# geometric_fba_sol = cobra.flux_analysis.geometric_fba(model)
# len(geometric_fba_sol.fluxes)

In [ ]:
for r in model.metabolites.get_by_id('mal__L_c').reactions:
    print(r.name, r)